data

In [1]:
import pandas as pd
from sklearn.metrics import classification_report

# Load the uploaded dataset
file_path = '../features/full_features_train.csv'
train_data = pd.read_csv(file_path)
# train_data = train_data[train_data['RL_num_19']>=12]
dev_data = pd.read_csv('../features/full_features_Dev.csv')
# dev_data = dev_data[dev_data['RL_num_19']>=12]

test_data = pd.read_csv('../features/full_features_Test.csv')
# test_data = test_data[test_data['RL_num_19']>=12]

# Check the unique values in the "Split" column
# data['Split'].unique()

# remove the irrelevant columns
train_data = train_data.drop(columns=['syllable_word', 'exception_lex', 'coordinated_verbs', 'words_max_level', 'levels_margin_0.02', 'vocab_samersubset_0.01', 'max_samerMSADA', 'words_max_level.1', 'levels_margin_0.02.1', 'vocab_samersubset_0.01.1', 'unique_words', 'max word count in RL', 'RL_num_7', 'RL_num_5', 'RL_num_3', 'd3lex_sents', 'lex_sents', 'd3tok_sents', 'word_sents', 'Sentence', 'Word Count', 'Readability Level', 'Annotator', 'annotation file', 'Project Phase', 'Source', 'Book', 'Author', 'Domain', 'Text Class', 'Reader Class', 'File', 'Split', 'Clean_Sentnece'])
dev_data = dev_data.drop(columns=['syllable_word', 'exception_lex', 'coordinated_verbs', 'unique_words', 'max word count in RL', 'RL_num_7', 'RL_num_5', 'RL_num_3', 'd3lex_sents', 'lex_sents', 'd3tok_sents', 'word_sents', 'Sentence', 'Word Count', 'Readability Level', 'Annotator', 'annotation file', 'Project Phase', 'Source', 'Book', 'Author', 'Domain', 'Text Class', 'Reader Class', 'File', 'Split', 'Clean_Sentnece'])

test_data = test_data.drop(columns=['words_max_level', 'levels_margin_0.02', 'vocab_samersubset_0.01', 'syllable_word', 'exception_lex', 'coordinated_verbs', 'unique_words', 'max word count in RL', 'RL_num_7', 'RL_num_5', 'RL_num_3', 'd3lex_sents', 'lex_sents', 'd3tok_sents', 'word_sents', 'Sentence', 'Word Count', 'Readability Level', 'Annotator', 'annotation file', 'Project Phase', 'Source', 'Book', 'Author', 'Domain', 'Text Class', 'Reader Class', 'File', 'Split', 'Clean_Sentnece'])
# add vocab_1.0 to test data
test_data['vocab_1.0'] = False

# relevant_columns = all columns except the target column "RL_num_19"
relevant_columns = train_data.columns.tolist()
relevant_columns.remove('RL_num_19')
relevant_columns.remove('ID')

X_train = train_data[relevant_columns]
y_train = train_data['RL_num_19']

X_dev = dev_data[relevant_columns]
y_dev = dev_data['RL_num_19']

X_test = test_data[relevant_columns]
y_test = test_data['RL_num_19']

X_train = X_train.fillna(X_train.mean())
X_dev = X_dev.fillna(X_dev.mean())
X_test = X_test.fillna(X_test.mean())

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# numerical columns
# num_cols = ['word count unique','syllables']
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist() 

X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_dev[num_cols] = scaler.transform(X_dev[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

X_train = X_train.fillna(0)

X_dev = X_dev.fillna(0)

X_test = X_test.fillna(0)

# true false columns
bool_cols = X_train.select_dtypes(include=['bool']).columns.tolist()
X_train[bool_cols] = X_train[bool_cols].astype(int)
X_dev[bool_cols] = X_dev[bool_cols].astype(int)
X_test[bool_cols] = X_test[bool_cols].astype(int)

# the rest of the columns, if value is 0 -> False if value is >0 -> True
# num_cols_remaining = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist() 
# # remove ['word count unique','syllables'] from the list
# num_cols.remove('word count unique')
# num_cols.remove('syllables')
# X_train[num_cols_remaining] = X_train[num_cols_remaining].applymap(lambda x: True if x > 0 else False)
# X_dev[num_cols_remaining] = X_dev[num_cols_remaining].applymap(lambda x: True if x > 0 else False)
# X_test[num_cols_remaining] = X_test[num_cols_remaining].applymap(lambda x: True if x > 0 else False)

In [16]:
len(dev_data)

8395

## Random Forest

In [11]:
# Train a RandomForest model on the training set
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [12]:
# Make predictions on Dev and Test datasets
y_pred_dev = model.predict(X_dev)
y_pred_test = model.predict(X_test)

In [13]:
# classification report
from sklearn.metrics import classification_report
print("Classification Report for Dev Set:")
print(classification_report(y_dev, y_pred_dev))

Classification Report for Dev Set:
              precision    recall  f1-score   support

          12       0.67      0.80      0.73      1945
          13       0.39      0.23      0.29       486
          14       0.54      0.52      0.53      1303
          15       0.20      0.26      0.22       302
          16       0.40      0.04      0.07       156
          17       0.33      0.08      0.13        48
          18       0.00      0.00      0.00        13
          19       0.00      0.00      0.00        15

    accuracy                           0.57      4268
   macro avg       0.32      0.24      0.25      4268
weighted avg       0.55      0.57      0.55      4268



/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [14]:
# save predictions to a file
dev_data['predicted_RL'] = y_pred_dev

In [15]:
# save just id, RL_num_19 and predicted_RL
dev_data[['ID', 'RL_num_19', 'predicted_RL']].to_csv('/Users/noor/Desktop/Thesis/Thesis/thesis_data/s31/conventional/random_forest/12_levels_dev_predictions.csv', index=False)

In [16]:
# classification report
from sklearn.metrics import classification_report
print("Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

Classification Report for Test Set:
              precision    recall  f1-score   support

          12       0.63      0.77      0.70      1555
          13       0.41      0.19      0.26       455
          14       0.51      0.56      0.53      1223
          15       0.23      0.33      0.27       356
          16       0.25      0.02      0.04       261
          17       0.14      0.01      0.02        87
          18       0.20      0.04      0.07        23
          19       0.00      0.00      0.00        15

    accuracy                           0.53      3975
   macro avg       0.30      0.24      0.24      3975
weighted avg       0.49      0.53      0.49      3975



/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [17]:
test_data['predicted_RL'] = y_pred_test

In [18]:
test_data[['ID', 'RL_num_19', 'predicted_RL']].to_csv('/Users/noor/Desktop/Thesis/Thesis/thesis_data/s31/conventional/random_forest/12_levels_test_predictions.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [ 350, 400],
    'max_depth': [40, 45, 50, 55],
    'min_samples_split': [15, 20, 22, 23, 25],
    'min_samples_leaf': [2, 4, 6, ],
    'criterion': ['gini', 'entropy']
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_params

# Train a RandomForest model with the best hyperparameters
best_model = RandomForestClassifier(**best_params)

In [17]:
best_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=40, min_samples_leaf=2, min_samples_split=15,
                       n_estimators=350)

In [18]:
# Make predictions on Dev and Test datasets
y_pred_dev_best = best_model.predict(X_dev)
y_pred_test_best = best_model.predict(X_test)

## Decision Tree

In [9]:
# train a decision tree model
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=30, random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=30, random_state=42)

In [10]:
# Make predictions on Dev and Test datasets
y_pred_dev = model.predict(X_dev)
y_pred_test = model.predict(X_test)

In [11]:
# classification report
print("Classification Report for Dev Set:")
print(classification_report(y_dev, y_pred_dev))

Classification Report for Dev Set:
              precision    recall  f1-score   support

           1       0.48      0.32      0.38        31
           2       0.35      0.46      0.40        41
           3       0.44      0.50      0.46       117
           4       0.21      0.41      0.28        63
           5       0.45      0.53      0.48       280
           6       0.39      0.49      0.43       188
           7       0.63      0.57      0.60       666
           8       0.79      0.64      0.71       740
           9       0.53      0.43      0.47       261
          10       0.81      0.65      0.72      1219
          11       0.42      0.67      0.52       521

    accuracy                           0.60      4127
   macro avg       0.50      0.52      0.50      4127
weighted avg       0.64      0.60      0.61      4127



In [12]:
dev_data['predicted_RL'] = y_pred_dev

dev_data[['ID', 'RL_num_19', 'predicted_RL']].to_csv('/Users/noor/Desktop/Thesis/Thesis/thesis_data/s31/conventional/decision tree/11_levels_dev_predictions.csv', index=False)

In [13]:
test_data['predicted_RL'] = y_pred_test
test_data[['ID', 'RL_num_19', 'predicted_RL']].to_csv('/Users/noor/Desktop/Thesis/Thesis/thesis_data/s31/conventional/decision tree/11_levelstest_predictions.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

# grid search for decision tree
param_grid = {
    'max_depth': [30, 35, 40, 45, 50],
    'min_samples_split': [15, 20, 22, 23, 25],
    'min_samples_leaf': [2, 4, 6, ],
    'criterion': ['gini', 'entropy']
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_params



In [25]:
# Train a DecisionTree model with the best hyperparameters
best_model = DecisionTreeClassifier(**best_params)

best_model.fit(X_train, y_train)

# Make predictions on Dev and Test datasets
y_pred_dev_best = best_model.predict(X_dev)
y_pred_test_best = best_model.predict(X_test)

# classification report
from sklearn.metrics import classification_report
print("Classification Report for Dev Set:")
print(classification_report(y_dev, y_pred_dev_best))

Classification Report for Dev Set:
              precision    recall  f1-score   support

          12       0.64      0.59      0.62      1945
          13       0.35      0.27      0.30       486
          14       0.55      0.48      0.51      1303
          15       0.09      0.27      0.14       302
          16       0.12      0.08      0.09       156
          17       0.33      0.08      0.13        48
          18       0.00      0.00      0.00        13
          19       0.00      0.00      0.00        15

    accuracy                           0.47      4268
   macro avg       0.26      0.22      0.22      4268
weighted avg       0.52      0.47      0.49      4268



/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

## SVM

In [2]:
from sklearn.metrics import classification_report


In [4]:
# train an svm model
from sklearn.svm import SVC
model = SVC(kernel='linear', C=1)
model.fit(X_train, y_train)

# Make predictions on Dev and Test datasets
y_pred_dev = model.predict(X_dev)
y_pred_test = model.predict(X_test)



In [5]:
# classification report
print("Classification Report SVM for Dev Set:")
print(classification_report(y_dev, y_pred_dev))

Classification Report SVM for Dev Set:
              precision    recall  f1-score   support

           1       0.30      0.23      0.26        31
           2       0.24      0.32      0.27        41
           3       0.42      0.50      0.45       117
           4       0.48      0.37      0.41        63
           5       0.39      0.56      0.46       280
           6       0.37      0.46      0.41       188
           7       0.51      0.57      0.54       666
           8       0.63      0.64      0.64       740
           9       0.58      0.34      0.43       261
          10       0.64      0.69      0.66      1219
          11       0.14      0.07      0.09       521
          12       0.44      0.53      0.48      1945
          13       0.36      0.17      0.24       486
          14       0.51      0.52      0.52      1301
          15       0.24      0.25      0.25       302
          16       0.42      0.03      0.06       156
          17       0.27      0.06      0.1

/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [7]:
dev_data['predicted_RL'] = y_pred_dev
dev_data[['ID', 'RL_num_19', 'predicted_RL']].to_csv('/Users/noor/Desktop/Thesis/Thesis/thesis_data/s31/conventional/SVM/11_levels_dev_predictions.csv', index=False)
test_data['predicted_RL'] = y_pred_test
test_data[['ID', 'RL_num_19', 'predicted_RL']].to_csv('/Users/noor/Desktop/Thesis/Thesis/thesis_data/s31/conventional/SVM/11_levels_test_predictions.csv', index=False)

In [3]:
# grid search to find the best hyperparameters
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid']
}
model = SVC()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_params

# Train an SVM model with the best hyperparameters
best_model = SVC(**best_params)
best_model.fit(X_train, y_train)

# Make predictions on Dev and Test datasets 
y_pred_dev = best_model.predict(X_dev)

# classification report
print("Classification Report for Dev Set:")
print(classification_report(y_dev, y_pred_dev))

Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time= 3.3min
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time= 3.4min
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time= 3.5min
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time= 7.5min
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time= 8.1min
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time= 8.2min
[CV] END ......................C=0.1, gamma=0.1, kernel=poly; total time= 3.8min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=11.0min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=11.3min
[CV] END ......................C=0.1, gamma=0.1, kernel=poly; total time= 3.4min
[CV] END ......................C=0.1, gamma=0.1, kernel=poly; total time= 3.4min
[CV] END .......................C=0.1, gamma=0.

/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [6]:
grid_search.best_params_

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

## BOOST

In [9]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.6 MB/s eta 0:00:00a 0:00:01


In [2]:
! brew install libomp

Running `brew update --auto-update`...


In [8]:
# XGBoost
from xgboost import XGBClassifier
model = XGBClassifier()
y_train = y_train -1
y_dev = y_dev -1
model.fit(X_train, y_train)

# Make predictions on Dev and Test datasets
y_pred_dev = model.predict(X_dev)
y_pred_test = model.predict(X_test)

# classification report
print("Classification Report for Dev Set:")
print(classification_report(y_dev, y_pred_dev))

Classification Report for Dev Set:
              precision    recall  f1-score   support

           0       0.50      0.48      0.49        31
           1       0.30      0.39      0.34        41
           2       0.47      0.51      0.49       117
           3       0.48      0.41      0.44        63
           4       0.40      0.54      0.46       280
           5       0.34      0.48      0.40       188
           6       0.51      0.57      0.54       666
           7       0.63      0.64      0.64       740
           8       0.60      0.41      0.49       261
           9       0.65      0.69      0.67      1219
          10       0.31      0.37      0.34       521
          11       0.46      0.47      0.47      1945
          12       0.38      0.25      0.30       486
          13       0.53      0.51      0.52      1303
          14       0.25      0.23      0.24       302
          15       0.30      0.06      0.11       156
          16       0.14      0.04      0.06   

In [9]:
dev_data['predicted_RL'] = y_pred_dev + 1
dev_data[['ID', 'RL_num_19', 'predicted_RL']].to_csv('/Users/noor/Desktop/Thesis/Thesis/thesis_data/s31/conventional/boost/dev_predictions.csv', index=False)
test_data['predicted_RL'] = y_pred_test + 1
test_data[['ID', 'RL_num_19', 'predicted_RL']].to_csv('/Users/noor/Desktop/Thesis/Thesis/thesis_data/s31/conventional/boost/test_predictions.csv', index=False)

In [10]:
len(dev_data)

8395

In [22]:
# get the repeated ids
repeated_ids = test_data[test_data.duplicated(subset=['ID'], keep=False)]['ID'].unique()

In [23]:
repeated_ids

array([53871, 67307, 55494, 67422])

In [24]:
dev_data[dev_data.duplicated(subset=['ID'], keep=False)]['ID'].unique()

array([37137, 58949])

In [4]:
# grid search to find the best hyperparameters
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [6, 8, 10, 12],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'colsample_bylevel': [0.5, 0.7, 0.9],
    'colsample_bynode': [0.5, 0.7, 0.9],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5, 2],
    'gamma': [0, 0.5, 1],
    'min_child_weight': [1, 5, 10]
}
model = XGBClassifier()

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_params


[CV] END colsample_bylevel=0.5, colsample_bynode=0.5, colsample_bytree=0.5, gamma=1, learning_rate=0.1, max_depth=8, min_child_weight=1, n_estimators=200, reg_alpha=0, reg_lambda=1, subsample=0.5; total time=   0.1s
[CV] END colsample_bylevel=0.5, colsample_bynode=0.5, colsample_bytree=0.5, gamma=1, learning_rate=0.1, max_depth=8, min_child_weight=1, n_estimators=200, reg_alpha=0, reg_lambda=1, subsample=0.5; total time=   0.0s
[CV] END colsample_bylevel=0.5, colsample_bynode=0.5, colsample_bytree=0.5, gamma=1, learning_rate=0.1, max_depth=8, min_child_weight=1, n_estimators=200, reg_alpha=0, reg_lambda=1, subsample=0.5; total time=   0.1s
[CV] END colsample_bylevel=0.5, colsample_bynode=0.5, colsample_bytree=0.5, gamma=1, learning_rate=0.1, max_depth=8, min_child_weight=1, n_estimators=200, reg_alpha=0, reg_lambda=1, subsample=0.7; total time=   0.0s
[CV] END colsample_bylevel=0.5, colsample_bynode=0.5, colsample_bytree=0.5, gamma=1, learning_rate=0.1, max_depth=8, min_child_weight=1,

KeyboardInterrupt: 

## Logistic Regression

In [9]:
len(y_dev)

8393

In [10]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions on Dev and Test datasets
y_pred_dev = model.predict(X_dev)
y_pred_test = model.predict(X_test)

# classification report
print("Classification Report for Dev Set:")
print(classification_report(y_dev, y_pred_dev))

Classification Report for Dev Set:
              precision    recall  f1-score   support

           1       0.30      0.10      0.15        31
           2       0.13      0.12      0.12        41
           3       0.37      0.53      0.44       117
           4       0.51      0.40      0.45        63
           5       0.38      0.54      0.44       280
           6       0.35      0.45      0.39       188
           7       0.51      0.57      0.54       666
           8       0.63      0.64      0.64       740
           9       0.65      0.36      0.46       261
          10       0.64      0.69      0.67      1219
          11       0.14      0.07      0.10       521
          12       0.44      0.54      0.49      1945
          13       0.36      0.22      0.28       486
          14       0.52      0.51      0.52      1301
          15       0.25      0.25      0.25       302
          16       0.40      0.04      0.07       156
          17       0.27      0.06      0.10   

/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [11]:
# grid search to find the best hyperparameters
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
model = LogisticRegression(max_iter=1000)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_params


Fitting 3 folds for each of 80 candidates, totalling 240 fits
[CV] END ....................C=0.1, penalty=l1, solver=lbfgs; total time=   0.1s
[CV] END ....................C=0.1, penalty=l1, solver=lbfgs; total time=   0.2s
[CV] END ................C=0.1, penalty=l1, solver=newton-cg; total time=   0.2s
[CV] END ....................C=0.1, penalty=l1, solver=lbfgs; total time=   0.2s
[CV] END ................C=0.1, penalty=l1, solver=newton-cg; total time=   0.2s
[CV] END ................C=0.1, penalty=l1, solver=newton-cg; total time=   0.2s
[CV] END ......................C=0.1, penalty=l1, solver=sag; total time=   0.1s
[CV] END ......................C=0.1, penalty=l1, solver=sag; total time=   0.1s
[CV] END ......................C=0.1, penalty=l1, solver=sag; total time=   0.1s
[CV] END ................C=0.1, penalty=l2, solver=newton-cg; total time=  24.6s
[CV] END ................C=0.1, penalty=l2, solver=newton-cg; total time=  27.1s
[CV] END ................C=0.1, penalty=l1, sol

/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END .....................C=0.1, penalty=l2, solver=saga; total time= 3.5min


/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


KeyboardInterrupt: 

In [15]:
# Linear Regression
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on Dev and Test datasets
y_pred_dev = model.predict(X_dev)
y_pred_test = model.predict(X_test)

# find the closest integer to the predicted value
y_pred_dev = [round(value) for value in y_pred_dev]
y_pred_test = [round(value) for value in y_pred_test]

# classification report
print("Classification Report for Dev Set:")
print(classification_report(y_dev, y_pred_dev))


Classification Report for Dev Set:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        41
           3       0.27      0.55      0.36       117
           4       0.05      0.02      0.02        63
           5       0.28      0.50      0.36       280
           6       0.16      0.11      0.13       188
           7       0.51      0.57      0.54       666
           8       0.63      0.64      0.64       740
           9       0.72      0.26      0.39       261
          10       0.63      0.70      0.66      1219
          11       0.14      0.13      0.14       521
          12       0.44      0.48      0.46      1945
          13       0.16      0.11      0.13       486
          14       0.48      0.46      0.47      1301
          15       0.29      0.18      0.22       302
          16       0.26      0.23      0.24       156
          17       0.27      0.06      0.10   

/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/noor/anaconda3/envs/camel_parser/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_